# Create hosted imagery layer from local raster raster (crf)

### Docs and resources
#### https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.utils.html#arcgis.raster.utils.generate_direct_access_url
#### https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.analytics.html#add_image
#### https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.analytics.html#copy-raster

## Import needed libraries

In [ ]:
import os
import arcpy
import getpass
import arcgis
from arcgis.raster import *
from arcgis.raster.analytics import *
from arcgis.gis import GIS

arcgis.env.verbose=True
arcgis.env.overwrite = True

import urllib3
urllib3.disable_warnings() 

## Provide username and password for publishing account

In [ ]:
 from credentials import username, password, org_url

## Connect to ArcGIS Online Account

In [ ]:
gis = GIS(org_url, username, password, verify_cert=True)
gis

## Specify the path to the CRF file
#### parse out what is needed

In [ ]:
upload_crf = r'E:\gis_lib\global_fishing_watch\v4\global_fish_watch_annual_fleet_sum_2016.crf'

### Extract the crf path, folder name, and the file name

crf_path = os.path.dirname(upload_crf)
print(crf_path)
# folder_name = "upload_crf.split('\\')[-2].replace('.crf','')"
folder_name = "imagery_upload"
print(folder_name)
crf_name = os.path.basename(upload_crf).replace('.crf','')
print(crf_name)

## generate the direct access url.
### this gives us a SAS token from Azure that allows us to decode and extract what is needed to create an ACS (Cloud Storage Connection)
### Request direct access URL from ArcGIS Online for the ArcGIS Online user that you are logged in as

In [ ]:
access_url = arcgis.raster.utils.generate_direct_access_url(expiration=1440, gis=gis)
access_url

## parse what we need from the access_url

In [ ]:
azure_account_name = access_url.split('//')[1].split('.')[0]
print("Azure account name: " + azure_account_name)
azure_container = access_url.split('?')[0].split('/')[-1]
print("Azure container name: " + azure_container)
azure_sas_token = access_url.split('?')[1].split('/')[-1]
print("Azure storage SAS Token: " + azure_sas_token)
azure_cpl_endpoint = access_url.split('/')[0] + "//" +  access_url.split('/',3)[-2]
print("Azure CPL Endpoint: " + azure_cpl_endpoint)

## Create the acs connection file locally

In [ ]:
acs_folder = r'E:\gis_lib\00_common\acs_connections'
target_connection_file = "agol_azure.acs"
acs_full_path = os.path.join(acs_folder, target_connection_file)

if os.path.exists(acs_full_path):
    os.remove(acs_full_path)

account_name = azure_account_name
container_name = azure_container
config_options = "AZURE_STORAGE_SAS_TOKEN " +  azure_sas_token + ";CPL_AZURE_ENDPOINT " + azure_cpl_endpoint

source_connection = arcpy.management.CreateCloudStorageConnectionFile(
    out_folder_path=acs_folder, 
    out_name=target_connection_file,
    service_provider="AZURE", 
    bucket_name=container_name, 
    access_key_id=account_name,
    config_options=config_options)

## Transfer local files to the Azure Blob Storage

In [ ]:
source = upload_crf
target = os.path.join(acs_folder, target_connection_file, folder_name, crf_name) + ".crf"
arcpy.management.TransferFiles(source, target)
print(target)
print(source)

## Direct upload image CRF to a single Tiled Imagery Layer

In [ ]:
data_url = access_url.split('?')[0] + "/" + folder_name + "/" + crf_name + '.crf'
print(data_url)

# layer_name = crf_name
layer_name = crf_name.replace('_fish_','_fishing_').replace('_2016','') + "_append_test2"

ImageryLayer = copy_raster(input_raster=data_url,
                           output_name=layer_name,
                           context={"compression":"LERC 0"},
                           gis=gis)

In [ ]:
ImageryLayer

In [ ]:
append_crf = r'E:\gis_lib\global_fishing_watch\v4\global_fish_watch_annual_fleet_sum_2017.crf'
append_crf_name = append_crf.split('\\')[-1]

print(append_crf_name)

target = os.path.join(acs_folder, target_connection_file, folder_name, append_crf_name)
arcpy.management.TransferFiles(append_crf, target)
print(target)
print(source)




In [ ]:
# arcgis.raster.analytics.add_image(ImageryLayer, [target])